In [1]:
import polars as pl

import rustystats as rs

data = pl.read_parquet("https://raw.githubusercontent.com/PricingFrontier/pricing-data-example/917c853e256df8d5814721ab56f72889a908bb08/data/processed/frequency_set.parquet")

train_data = data.filter(pl.col("Group") <= "3")
test_data = data.filter(pl.col("Group") == "4")
holdout_data = data.filter(pl.col("Group") == "5")

In [2]:
terms = {
        'BonusMalus': {'type': 'ns'}, 
        'VehAge': {'type': 'ns'},
        'DrivAge': {'type': 'ns'}, 
        'VehPower': {'type': 'ns'}, 
        'Area': {'type': 'target_encoding'},
        'Region': {'type': 'target_encoding'},
        'VehGas': {'type': 'target_encoding'},
        'VehBrand': {'type': 'target_encoding'}
    }

interactions=[{
        "VehPower": {"type": "linear"},
        "Region": {"type": "categorical"},
        "target_encoding": True,
        "include_main": False
    }]

result = rs.glm_dict(
    response='ClaimCount',
    terms=terms,
    interactions=interactions,
    data=data,
    family='poisson',
    offset ='Exposure'
).fit()

In [3]:
#writes analysis/diagnostics
result.diagnostics(
    train_data=train_data,
    test_data = train_data,
    categorical_factors=["Region", "Area", "VehBrand", "VehGas"],
    continuous_factors=["VehPower", "VehAge", "DrivAge", "BonusMalus" ]
)

ModelDiagnostics(model_summary={'formula': 'ClaimCount ~ ns(BonusMalus, df=10) + ns(VehAge, df=10) + ns(DrivAge, df=10) + ns(VehPower, df=10) + TE(Area) + TE(Region) + TE(VehGas) + TE(VehBrand)', 'family': 'poisson', 'link': 'log', 'n_obs': 406439, 'n_params': 42, 'df_resid': 406397, 'converged': True, 'iterations': 7}, train_test=TrainTestComparison(train=DatasetDiagnostics(dataset='train', n_obs=406439, total_exposure=214930.9, total_actual=21408.0, total_predicted=21173.52, loss=0.30952, deviance=125800.96, log_likelihood=-83674.3, aic=167432.61, gini=0.2841, auc=0.642, ae_ratio=1.0111, ae_by_decile=[DecileMetrics(decile=1, n=40643, exposure=1926.83, actual=0.38872, predicted=0.074826, ae_ratio=5.195), DecileMetrics(decile=2, n=40643, exposure=4244.72, actual=0.224278, predicted=0.088201, ae_ratio=2.5428), DecileMetrics(decile=3, n=40643, exposure=8295.52, actual=0.163703, predicted=0.083931, ae_ratio=1.9504), DecileMetrics(decile=4, n=40643, exposure=14078.38, actual=0.119048, pred

In [4]:
print(result.summary())

                                 GLM Results                                  

Family:              Poisson         No. Observations:        678012
Link Function:       (default)       Df Residuals:            677970
Method:              IRLS + Smooth   Df Model:                    41
Scale:               1.0000          Alpha (λ):               0.0000
L1 Ratio:            0.00            Iterations:                   7
Non-zero coefs:      41             

Log-Likelihood:         -139167.5580 Deviance:                209334.9698
AIC:                     278419.1159 Null Deviance:           220244.0296
BIC:                     278899.0466 Pearson chi2:             1515306.86
Converged:           True           

------------------------------------------------------------------------------
Variable                  Coef    Std.Err        z    P>|z|                 95% CI     
------------------------------------------------------------------------------
Intercept              -3.1738 

In [5]:
result.predict(holdout_data)

array([0.05146307, 0.04830934, 0.04926051, ..., 0.00556822, 0.01738246,
       0.07078242], shape=(135957,))

In [6]:
# Model spec before interaction
terms = {
    "BonusMalus": {"type": "linear"},
    "VehAge": {"type": "ns", "df": 5},
    "DrivAge": {"type": "linear"},
    "Region": {"type": "target_encoding"},
    "VehBrand": {"type": "target_encoding"},
    "Area": {"type": "target_encoding"},
}
# Interaction to add (DrivAge x BonusMalus)
interactions = [
    {
        "DrivAge": {"type": "linear"},
        "Region": {"type": "target_encoding"},
        "include_main": False,
    }
]
# Fit with interaction
result = rs.glm_dict(
    response="ClaimCount",
    terms=terms,
    interactions=interactions,
    data=train_data,
    family="poisson",
    offset="Exposure",
).fit()


In [7]:
print(result.summary())

                                 GLM Results                                  

Family:              Poisson         No. Observations:        406439
Link Function:       (default)       Df Residuals:            406428
Method:              IRLS            Df Model:                    10
Scale:               1.0000          Iterations:                   6

Log-Likelihood:          -84128.1572 Deviance:                126708.6648
AIC:                     168278.3145 Null Deviance:           132328.3090
BIC:                     168398.3816 Pearson chi2:              934820.68
Converged:           True           

------------------------------------------------------------------------------
Variable                 Coef    Std.Err        z    P>|z|                 95% CI     
------------------------------------------------------------------------------
Intercept             -4.3595     0.1608  -27.104  <0.0001   [ -4.6747,  -4.0442]  ***
BonusMalus             0.0220     0.0004   54.692  

In [8]:
#creates analysis/exploration.json
exploration = rs.explore_data(
    data=train_data,
    response="ClaimCount",
    categorical_factors=["Region", "Area", "VehBrand", "VehGas"],
    continuous_factors=["VehPower", "VehAge", "DrivAge", "BonusMalus" ],
    exposure="Exposure",
)

In [9]:
#writes analysis/diagnostics
result.diagnostics(
    train_data=train_data,
    test_data = test_data,
    categorical_factors=["Region", "Area", "VehBrand", "VehGas"],
    continuous_factors=["VehPower", "VehAge", "DrivAge", "BonusMalus" ]
)

ModelDiagnostics(model_summary={'formula': 'ClaimCount ~ BonusMalus + ns(VehAge, df=5) + DrivAge + TE(Region) + TE(VehBrand) + TE(Area)', 'family': 'poisson', 'link': 'log', 'n_obs': 406439, 'n_params': 11, 'df_resid': 406428, 'converged': True, 'iterations': 6}, train_test=TrainTestComparison(train=DatasetDiagnostics(dataset='train', n_obs=406439, total_exposure=214930.9, total_actual=21408.0, total_predicted=21508.34, loss=0.311696, deviance=126685.58, log_likelihood=-84116.61, aic=168255.23, gini=0.2597, auc=0.6298, ae_ratio=0.9953, ae_by_decile=[DecileMetrics(decile=1, n=40643, exposure=1903.56, actual=0.412386, predicted=0.078271, ae_ratio=5.2687), DecileMetrics(decile=2, n=40643, exposure=4176.36, actual=0.235612, predicted=0.091791, ae_ratio=2.5668), DecileMetrics(decile=3, n=40643, exposure=8108.11, actual=0.16576, predicted=0.087868, ae_ratio=1.8865), DecileMetrics(decile=4, n=40643, exposure=13734.89, actual=0.127267, predicted=0.083351, ae_ratio=1.5269), DecileMetrics(decile

In [10]:
predictions = result.predict(holdout_data)

In [11]:
model_bytes = result.to_bytes()
with open("model.bin", "wb") as f:
    f.write(model_bytes)

In [12]:
with open("model.bin", "rb") as f:
    loaded = rs.GLMModel.from_bytes(f.read())

In [13]:
predictions2 = loaded.predict(holdout_data)

In [14]:
import numpy as np
print(np.array_equal(predictions2, predictions))

True


In [15]:
#writes analysis/diagnostics
loaded.diagnostics(
    train_data=train_data,
    test_data = train_data,
    categorical_factors=["Region", "Area", "VehBrand", "VehGas"],
    continuous_factors=["VehPower", "VehAge", "DrivAge", "BonusMalus" ]
)

ModelDiagnostics(model_summary={'formula': 'ClaimCount ~ BonusMalus + ns(VehAge, df=5) + DrivAge + TE(Region) + TE(VehBrand) + TE(Area)', 'family': 'poisson', 'link': 'log', 'n_obs': 406439, 'n_params': 11, 'df_resid': 406428, 'converged': True, 'iterations': 6}, train_test=TrainTestComparison(train=DatasetDiagnostics(dataset='train', n_obs=406439, total_exposure=214930.9, total_actual=21408.0, total_predicted=21508.34, loss=0.311696, deviance=126685.58, log_likelihood=-84116.61, aic=168255.23, gini=0.2597, auc=0.6298, ae_ratio=0.9953, ae_by_decile=[DecileMetrics(decile=1, n=40643, exposure=1903.56, actual=0.412386, predicted=0.078271, ae_ratio=5.2687), DecileMetrics(decile=2, n=40643, exposure=4176.36, actual=0.235612, predicted=0.091791, ae_ratio=2.5668), DecileMetrics(decile=3, n=40643, exposure=8108.11, actual=0.16576, predicted=0.087868, ae_ratio=1.8865), DecileMetrics(decile=4, n=40643, exposure=13734.89, actual=0.127267, predicted=0.083351, ae_ratio=1.5269), DecileMetrics(decile

In [16]:
result2 = rs.glm_dict(
    response="ClaimCount",
    terms={
        "Region": {"type": "target_encoding"},
        "BonusMalus": {"type": "linear"},
        "BonusMalus2": {"type": "expression", "expr": "BonusMalus ** 2"}
    },
    data=train_data,
    family="poisson",
    offset="Exposure",
).fit(
    regularization="elastic_net"
)

In [17]:
#writes analysis/diagnostics
result2.diagnostics(
    train_data=train_data,
    test_data = train_data,
    categorical_factors=["Region", "Area", "VehBrand", "VehGas"],
    continuous_factors=["VehPower", "VehAge", "DrivAge", "BonusMalus" ]
)

ModelDiagnostics(model_summary={'formula': 'ClaimCount ~ TE(Region) + BonusMalus + I(BonusMalus ** 2)', 'family': 'poisson', 'link': 'log', 'n_obs': 406439, 'n_params': 4, 'df_resid': 406435, 'converged': True, 'iterations': 6}, train_test=TrainTestComparison(train=DatasetDiagnostics(dataset='train', n_obs=406439, total_exposure=214930.9, total_actual=21408.0, total_predicted=21400.88, loss=0.31867, deviance=129520.08, log_likelihood=-85533.87, aic=171075.73, gini=0.1835, auc=0.5917, ae_ratio=1.0003, ae_by_decile=[DecileMetrics(decile=1, n=40643, exposure=1695.69, actual=0.641038, predicted=0.088891, ae_ratio=7.2115), DecileMetrics(decile=2, n=40643, exposure=3745.58, actual=0.288073, predicted=0.099112, ae_ratio=2.9065), DecileMetrics(decile=3, n=40643, exposure=7518.28, actual=0.182888, predicted=0.094992, ae_ratio=1.9253), DecileMetrics(decile=4, n=40643, exposure=12503.62, actual=0.13884, predicted=0.095059, ae_ratio=1.4606), DecileMetrics(decile=5, n=40643, exposure=18493.77, actu

In [18]:
result3 = rs.glm_dict(
    response="ClaimCount",
    terms={
        "BonusMalus": {"type": "ns"},
        "VehAge": {"type": "ns"},
        "DrivAge": {"type": "linear"},
        "VehBrand": {"type": "target_encoding"}
    },
    interactions= [
        {
            "VehAge": {"type": "linear"},
            "BonusMalus": {"type": "linear"},
            "include_main": False
        },
        {
            "DrivAge": {"type": "linear"},
            "BonusMalus": {"type": "linear"},
            "include_main": False
        }
    ],
    data=train_data,
    family="poisson",
    offset="Exposure",
).fit(
    regularization="elastic_net",
    include_unregularized=False
)

In [19]:
import rustystats as rs

# Model spec that caused the issue when elastic_net was applied
terms = {
    "BonusMalus": {'type' : "linear"},
    "VehAge": {'type' : "ns"}, 
    "DrivAge": {'type' : "ns"},
    "Area": {'type' : "target_encoding"},
}

interactions= [
        {
            "DrivAge": {"type": "linear"},
            "BonusMalus": {"type": "linear"},
            "include_main": False
        }
    ]

# Build the model
glm = rs.glm_dict(
    response="ClaimCount",
    terms=terms,
    interactions=interactions,
    data=train_data,  # freMTPL2freq dataset
    family="poisson",
    offset="Exposure",  # log-linked
)

# This is where the issue occurs - regularized fit + diagnostics
result = glm.fit()

# Check if diagnostics returns None for deviance
diag = result.diagnostics(train_data=train_data, test_data=holdout_data)
print(diag.to_dict())  # Check train_test.test.deviance

{'model_summary': {'formula': 'ClaimCount ~ BonusMalus + ns(VehAge, df=10) + ns(DrivAge, df=10) + TE(Area)', 'family': 'poisson', 'link': 'log', 'n_obs': 406439, 'n_params': 22, 'df_resid': 406417, 'converged': True, 'iterations': 7}, 'train_test': {'train': {'dataset': 'train', 'n_obs': 406439, 'total_exposure': 214930.9, 'total_actual': 21408.0, 'total_predicted': 21445.48, 'loss': 0.310817, 'deviance': 126328.34, 'log_likelihood': -83937.99, 'aic': 167919.99, 'gini': 0.2737, 'auc': 0.6368, 'ae_ratio': 0.9983, 'ae_by_decile': [{'decile': 1, 'n': 40643, 'exposure': 1903.5, 'actual': 0.397689, 'predicted': 0.077513, 'ae_ratio': 5.1306}, {'decile': 2, 'n': 40643, 'exposure': 4159.16, 'actual': 0.239952, 'predicted': 0.090977, 'ae_ratio': 2.6375}, {'decile': 3, 'n': 40643, 'exposure': 8131.24, 'actual': 0.164551, 'predicted': 0.08707, 'ae_ratio': 1.8899}, {'decile': 4, 'n': 40643, 'exposure': 13707.47, 'actual': 0.123801, 'predicted': 0.082797, 'ae_ratio': 1.4952}, {'decile': 5, 'n': 406